Задание 1.
Выберите датасет, который имеет отношение к вашей области интересов или исследований. Датасет должен содержать неструктурированные данные, требующие разметки для решения конкретной задачи, например, анализа настроений или распознавания именованных сущностей.

Задание 2.
Выполните разметку на основе правил (rule-based labeling) на подмножестве выбранного датасета. Разработайте и реализуйте набор правил или условий, которые позволят автоматически присваивать метки данным на основе определенных шаблонов или критериев.

Задача 3.
Выполните разметку вручную отдельного подмножества выбранного датасета с помощью выбранного вами инструмента разметки.

Задача 4.
Объедините данные, размеченные вручную, с данными, размеченными на основе правил. Объедините два подмножества размеченных данных в один набор данных, сохранив при этом соответствующую структуру и целостность.

Задача 5.
Обучите модель машинного обучения, используя объединенный набор размеченных данных. Разделите датасет на обучающий и тестовый наборы и используйте обучающий набор для обучения модели.

Задача 6.
Оценить эффективность обученной модели на тестовом датасете. Используйте подходящие метрики оценки. Интерпретируйте результаты и проанализируйте эффективность модели в решении задачи разметки.
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

С помощью xpath со страницы https://en.wikipedia.org/wiki/Glossary_of_computer_science получены термины компьютерных наук (файл comp_sci_terms.csv).

Взят список названий статей естественных и компьютерных наук papers.csv. Добавлена колонка Computer Science, в которой 1 означает Computer Science, а 0 - Natural Science.

In [1]:
import pandas as pd

papers = pd.read_csv("papers.csv", delimiter=';')

In [2]:
papers.head()

,TITLE,Computer Science
0,Reconstructing Subject-Specific Effect Maps,NaN
1,Rotation Invariance Neural Network,NaN
2,Spherical polyharmonics and Poisson kernels fo...,NaN
3,A finite element approximation for the stochas...,NaN
4,Comparative study of Discrete Wavelet Transfor...,NaN


In [3]:
papers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20719 entries, 0 to 20718
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TITLE             20719 non-null  object 
 1   Computer Science  0 non-null      float64
dtypes: float64(1), object(1)
memory usage: 323.9+ KB


In [4]:
comp_sci_terms = pd.read_csv("comp_sci_terms.csv", delimiter=';')
comp_sci_terms.head()

,TERMS
0,abstract data type
1,adt
2,abstract method
3,abstraction
4,agent architecture


In [5]:
def rule_based_labeling(row):
    row = row.lower()
    for term in comp_sci_terms['TERMS']:
        if term in row:
            return 1 
    return 0

In [6]:
papers['Computer Science'] = papers['TITLE'].apply(rule_based_labeling)
papers.head()

,TITLE,Computer Science
0,Reconstructing Subject-Specific Effect Maps,0
1,Rotation Invariance Neural Network,1
2,Spherical polyharmonics and Poisson kernels fo...,1
3,A finite element approximation for the stochas...,0
4,Comparative study of Discrete Wavelet Transfor...,1


In [7]:
papers.to_csv('papers_rule_lbls.csv', index=False, sep=';')

In [8]:
# 100 первых размечено вручную в табличном редакторе
papers_man = pd.read_csv("papers_rule_lbls_100mn.csv", delimiter=';')
papers_man.head()

,TITLE,Computer Science
0,Reconstructing Subject-Specific Effect Maps,1
1,Rotation Invariance Neural Network,1
2,Spherical polyharmonics and Poisson kernels fo...,0
3,A finite element approximation for the stochas...,0
4,Comparative study of Discrete Wavelet Transfor...,0


In [9]:
# 20% на обучение
papers_train = papers_man[:4000]
papers_test = papers_man[4000:]
papers_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   TITLE             4000 non-null   object
 1   Computer Science  4000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 62.6+ KB


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import numpy as np

In [11]:
# Определение функции для обучения модели логистической регрессии на размеченных данных
def train_model(labeled_papers):
    # Векторизация текстовых данных с помощью TF-IDF
    vectorizer = TfidfVectorizer()
    x = vectorizer.fit_transform(labeled_papers['TITLE'])
    y = labeled_papers['Computer Science']

    # Обучение модели логистической регрессии на размеченных данных
    model = LogisticRegression()
    model.fit(x, y)

    return model, vectorizer

In [12]:
# Обучение начальной модели на небольшом наборе 
model, vectorizer = train_model(papers_train)

In [13]:
x_unlabeled = vectorizer.transform(papers_test['TITLE'])
y_unlabeled_predicted = model.predict(x_unlabeled)

In [14]:
# Оценка модели на тестовом датасете
X_test = vectorizer.transform(papers_test['TITLE'])
y_test_predicted = model.predict(X_test)
f1 = f1_score(papers_test['Computer Science'], y_test_predicted)

print(f1)

0.7742705845706572


Соответствие предсказаний в 77% достаточно неплохое. Его можно было бы улучшить за счет увеличения количества пунктов с ручной разметкой или же за счет более гибкой разметки по правилам. Для улучшения разметки по правилам можно добавить словарь терминов естественных наук и сравнивать количество терминов обоих вариантов. Более точное предсказание дало бы использование аннотаций к статьям.